In [3]:
#Importing dependencies
import pandas as pd
from sodapy import Socrata
import scipy.stats as st
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

# Importing data for API call
from config import app_token
from config import username
from config import password

In [4]:
# Getting the data from API NY Open Data for the first database: 
client = Socrata("data.cityofnewyork.us",
                  app_token, username,
                  password)
# Database returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("gaq9-z3hz", limit=2832)

# Convert to pandas DataFrame
Recycling_Div_Captures = pd.DataFrame.from_records(results)
Recycling_Div_Captures

,_zone,district,fiscal_month_number,fiscal_year,month_name,diversion_rate_total_total_recycling_total_waste_,capture_rate_paper_total_paper_max_paper_,capture_rate_mgp_total_mgp_max_mgp_,capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100
0,Brooklyn North,BKN01,10,2019,April,14.6870926033314,44.9091597758182,43.0340618265488,44.1467643388238
1,Brooklyn North,BKN02,10,2019,April,19.9501814568266,34.1940201111571,57.9470313498761,41.2136999269730
2,Brooklyn North,BKN03,10,2019,April,12.1641613296565,33.5215567195417,44.9197311047560,38.1559365633997
3,Brooklyn North,BKN04,10,2019,April,15.5418031271589,35.2113607057133,68.5112600518364,48.7507554470644
4,Brooklyn North,BKN05,10,2019,April,10.0518454148520,22.2654301541493,45.0517913754661,31.5301290076712
...,...,...,...,...,...,...,...,...,...
2827,Queens West,QW06,3,2016,September,20.1214735183925,30.4196893217642,67.9561359185218,39.0333079418224
2828,Queens West,QW09,3,2016,September,17.4348618222880,41.1065470054454,79.6950721141275,54.2727362237205
2829,Staten Island,SI01,3,2016,September,18.6838350777916,39.5199429139324,71.7164916919349,49.6526689393673
2830,Staten Island,SI02,3,2016,September,19.0168187778538,44.4579262409406,74.9601995452711,54.0574389302287


In [5]:
# creating path for the citywide subsort 
data_path = "Resources/Citywide_Subsort.csv"
# Read the data into df
cws_df = pd.read_csv(data_path)
#Preview data
cws_df.head()

,Material,Aggregate Percent,Refuse Percent,MGP Percent,Paper Percent,Organic Percent,Material Group,DSNY Diversion Summary Category,Location
0,Newspaper,0.019,0.011,0.005,0.098,0.000,Paper,Designated Paper,Citywide
1,Plain OCC/Kraft Paper,0.056,0.012,0.010,0.464,0.002,Paper,Designated Paper,Citywide
2,Paper Bags: Kraft Grocery,0.002,0.001,0.000,0.008,0.000,Paper,Designated Paper,Citywide
3,Paper Bags: Non-food Retail,0.002,0.002,0.000,0.008,0.000,Paper,Designated Paper,Citywide
4,Paper Bags: Fast Food Bags,0.001,0.002,0.000,0.001,0.000,Paper,Designated Paper,Citywide


In [6]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\samue\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [7]:
url = 'https://data.cityofnewyork.us/City-Government/DSNY-Waste-Characterization-Mainsort/k3ks-jzek'
browser.visit(url)

In [8]:
df_list = []
for x in range(1,41):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find_all('div', class_='socrata-visualization')
    df_list.append(pd.read_html(str(table))[0]) 
    button = browser.find_by_xpath('//*[@id="app"]/div/div[2]/section[4]/div[2]/div/div/div[4]/div[2]/span[1]/button[2]').click()

In [9]:
df_combined = pd.concat(df_list)

In [10]:
df_combined.reset_index(drop=True, inplace=True)

In [11]:
cwm_df = df_combined
cwm_df.head()

,Material,Aggregate Percent,Refuse Percent,MGP Percent,Paper Percent,Organic Percent,Material Group,DSNY Diversion Summary Category,Location
0,Newspaper,1.9%,1.1%,0.5%,9.8%,0%,Paper,Designated Paper,Citywide
1,Plain OCC/Kraft Paper,5.7%,1.4%,1%,47.2%,0.2%,Paper,Designated Paper,Citywide
2,High Grade Paper,0.7%,0.5%,0.1%,2.6%,0%,Paper,Designated Paper,Citywide
3,Mixed Low Grade Paper,8.7%,6.8%,2.4%,31.4%,0.9%,Paper,Designated Paper,Citywide
4,Paper: Compostable/Soiled/Waxed OCC/Kraft,8.1%,9.8%,0.7%,1.4%,2%,Paper,Organics Suitable for Composting,Citywide


In [12]:
# Analizing material groups
unique_mat_subsort = cws_df["Material Group"].unique()
print(f'Material groups (subsort): {unique_mat_subsort}')
unique_mat_mainsort = cwm_df["Material Group"].unique()
print(f'Material groups (mainsort): {unique_mat_mainsort}')

Material groups (subsort): ['Paper' 'Plastic' 'Glass' 'Metal' 'Organic' 'E-Waste' 'C&D'
 'Special Waste' 'Miscellaneous Inorganics']
Material groups (mainsort): ['Paper' 'Plastic' 'Glass' 'Metal' 'Organic' 'E-Waste' 'C&D'
 'Special Waste' 'Miscellaneous Inorganics']


In [13]:
unique_loc_subsort = cws_df["Location"].unique()
print(f"Locations register in the Subsort dataset: {unique_loc_subsort}")
print("----------------------------------------------------------------")
unique_loc_mainsort = cwm_df["Location"].unique()
print(f"Locations register in the Mainsort dataset: {unique_loc_mainsort}")
print("----------------------------------------------------------------")
uniqie_loc_general = Recycling_Div_Captures["_zone"].unique()
print(f"Locations register in the general dataset: {uniqie_loc_general}")

Locations register in the Subsort dataset: ['Citywide' 'Brooklyn' 'Queens' 'Manhattan' 'Staten Island' 'Bronx'
 'Schools' 'NYCHA']
----------------------------------------------------------------
Locations register in the Mainsort dataset: ['Citywide' 'Brooklyn' 'Queens' 'Manhattan' 'Staten Island' 'Bronx'
 'Schools' 'NYCHA']
----------------------------------------------------------------
Locations register in the general dataset: ['Brooklyn North' 'Brooklyn South' 'Bronx' 'Manhattan' 'Queens East'
 'Queens West' 'Staten Island']


In [14]:
# Format change in the general dataset
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Brooklyn North', r'Brooklyn')
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Brooklyn South', r'Brooklyn')
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Queens East', r'Queens')
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Queens West', r'Queens')

In [15]:
# First filter: Recycling_Div_Captures data should be filter just for 2017 year
Recycling_Div_Captures["fiscal_year"] = Recycling_Div_Captures["fiscal_year"].apply(pd.to_numeric, errors='coerce')
Recycling_Div_Captures = Recycling_Div_Captures.loc[Recycling_Div_Captures["fiscal_year"]==2017,:]

In [16]:
# Rename columns and convert to_numeric 
Recycling_Div_Captures = Recycling_Div_Captures.rename(columns={
    "diversion_rate_total_total_recycling_total_waste_": "Total recycling rate",
    "capture_rate_paper_total_paper_max_paper_": "Paper rate",
    "capture_rate_mgp_total_mgp_max_mgp_": "MGP rate"
})
cols = ["Total recycling rate", "Paper rate", "MGP rate"]
Recycling_Div_Captures[cols] = Recycling_Div_Captures[cols].apply(pd.to_numeric, errors='coerce')

In [17]:
# Subsort percentages format 
cols = ["Aggregate Percent", "Refuse Percent", "MGP Percent", "Paper Percent", "Organic Percent"]
cws_df[cols] = cws_df[cols] *100
cws_df.head()

,Material,Aggregate Percent,Refuse Percent,MGP Percent,Paper Percent,Organic Percent,Material Group,DSNY Diversion Summary Category,Location
0,Newspaper,1.9,1.1,0.5,9.8,0.0,Paper,Designated Paper,Citywide
1,Plain OCC/Kraft Paper,5.6,1.2,1.0,46.4,0.2,Paper,Designated Paper,Citywide
2,Paper Bags: Kraft Grocery,0.2,0.1,0.0,0.8,0.0,Paper,Designated Paper,Citywide
3,Paper Bags: Non-food Retail,0.2,0.2,0.0,0.8,0.0,Paper,Designated Paper,Citywide
4,Paper Bags: Fast Food Bags,0.1,0.2,0.0,0.1,0.0,Paper,Designated Paper,Citywide


In [18]:
# Mainsort percentages format 
cols = ["Aggregate Percent", "Refuse Percent", "MGP Percent", "Paper Percent", "Organic Percent"]
cwm_df["Aggregate Percent"]= cwm_df["Aggregate Percent"].str.replace(r'%', r'')
cwm_df["Refuse Percent"]= cwm_df["Refuse Percent"].str.replace(r'%', r'')
cwm_df["MGP Percent"]= cwm_df["MGP Percent"].str.replace(r'%', r'')
cwm_df["Paper Percent"]= cwm_df["Paper Percent"].str.replace(r'%', r'')
cwm_df["Organic Percent"]= cwm_df["Organic Percent"].str.replace(r'%', r'')

cwm_df[cols]= cwm_df[cols].apply(pd.to_numeric, errors='coerce')
print(cwm_df.dtypes)
cwm_df.head()

Material                            object
Aggregate Percent                  float64
Refuse Percent                     float64
MGP Percent                        float64
Paper Percent                      float64
Organic Percent                    float64
Material Group                      object
DSNY Diversion Summary Category     object
Location                            object
dtype: object


,Material,Aggregate Percent,Refuse Percent,MGP Percent,Paper Percent,Organic Percent,Material Group,DSNY Diversion Summary Category,Location
0,Newspaper,1.9,1.1,0.5,9.8,0.0,Paper,Designated Paper,Citywide
1,Plain OCC/Kraft Paper,5.7,1.4,1.0,47.2,0.2,Paper,Designated Paper,Citywide
2,High Grade Paper,0.7,0.5,0.1,2.6,0.0,Paper,Designated Paper,Citywide
3,Mixed Low Grade Paper,8.7,6.8,2.4,31.4,0.9,Paper,Designated Paper,Citywide
4,Paper: Compostable/Soiled/Waxed OCC/Kraft,8.1,9.8,0.7,1.4,2.0,Paper,Organics Suitable for Composting,Citywide


In [19]:
# Paper collection
# Materials paper types register in subsort
paper_material_sub = cws_df.loc[cws_df["Material Group"]=="Paper",["Material"]]
paper_material_sub = paper_material_sub["Material"].unique()
set_sub = set(paper_material_sub)

# Organic collection
# Materials Organic types register in subsort
org_material_sub = cws_df.loc[cws_df["Material Group"]=="Organic",["Material"]]
org_material_sub = org_material_sub["Material"].unique()
set_sub_o = set(org_material_sub)

# E-waste collection
# Materials ewaste types register in subsort
ew_material_sub = cws_df.loc[cws_df["Material Group"]=="E-Waste",["Material"]]
ew_material_sub = ew_material_sub["Material"].unique()
set_sub_e = set(ew_material_sub)

In [20]:
# Materials paper types register in mainsort
paper_material_main = cwm_df.loc[cws_df["Material Group"]=="Paper",["Material"]]
paper_material_main = paper_material_main["Material"].unique()
set_main = set(paper_material_main)

# Materials paper types register in mainsort
org_material_main = cwm_df.loc[cws_df["Material Group"]=="Organic",["Material"]]
org_material_main = org_material_main["Material"].unique()
set_main_o = set(org_material_main)

# Materials paper types register in mainsort
ew_material_main = cwm_df.loc[cws_df["Material Group"]=="E-Waste",["Material"]]
ew_material_main = ew_material_main["Material"].unique()
set_main_e = set(ew_material_main)

In [21]:
# Mergin material without duplication 
no_duplicates = list(set_main-set_sub)
paper_material_sub = list(set_sub)
paper_material_final = paper_material_sub + no_duplicates
paper_material_final

no_duplicates_o = list(set_main_o-set_sub_o)
org_material_sub = list(set_sub_o)
org_material_final = org_material_sub + no_duplicates_o
print(org_material_final)

no_duplicates_e = list(set_main_e-set_sub_e)
ew_material_sub = list(set_sub_e)
ew_material_final = ew_material_sub + no_duplicates_e
print(ew_material_final)

['Shoes/Rubber/Leather', 'Animal By-Products', 'Disposable Diapers & Sanitary Products', 'Carpet/Upholstery', 'Garden Hoses', 'Miscellaneous Organics', 'Textiles:  Clothing', 'Yard Waste', 'Fines', 'Textiles:  Non-clothing', 'Non-C&D Wood', 'Food', 'Compressed Gas Cylinders/Fire Extinguishers', 'Latex Paint', 'Pesticides/Herbicides/Rodenticides', 'Oil-Based Paint/Solvent', 'Other Potentially Harmful Wastes', 'Computer Monitors', 'Fluorescent Tubes/CFLs', 'Textiles: Clothing', 'Audio/Visual Equipment-TV Peripherals (Covered)', 'Dry-Cell Batteries', 'Mercury-Laden Wastes', 'Disposable Diapers/Sanitary Products', 'Audio/Visual Equipment-Other (Non-Covered)', 'Newspaper', 'Plain OCC/Kraft Paper', 'Televisions', 'Home Medical Products', 'Miscellaneous Inorganics', 'Other Computer Equipment']
['Audio/Visual Equipment - TV Peripherals (Covered)', 'Audio/Visual Equipment - Other (Non-covered)', 'Computer Monitors', 'Other Computer Equipment', 'Televisions', 'Mixed Low Grade Paper', 'Other C&D 

In [22]:
# Filter just papper material group subsort
paper_sub_df = cws_df.loc[cws_df["Material Group"]=="Paper",:]
# Filter just papper material group mainsort
paper_main_df = cwm_df.loc[cws_df["Material Group"]=="Paper",:]


# Filter just organic material group subsort
org_sub_df = cws_df.loc[cws_df["Material Group"]=="Organic",:]
# Filter just organic material group mainsort
org_main_df = cwm_df.loc[cws_df["Material Group"]=="Organic",:]

# Filter just e-waste material group subsort
ew_sub_df = cws_df.loc[cws_df["Material Group"]=="E-Waste",:]
# Filter just e-waste material group mainsort
ew_main_df = cwm_df.loc[cws_df["Material Group"]=="E-Waste",:]

# Group and calculate the mean of aggregate and refuse percentage per location
agg_subsort = paper_sub_df.groupby("Location")["Aggregate Percent"].mean()
refuse_subsort = paper_sub_df.groupby("Location")["Refuse Percent"].mean()
agg_mainsort = paper_main_df.groupby("Location")["Aggregate Percent"].mean()
refuse_mainsort = paper_main_df.groupby("Location")["Refuse Percent"].mean()


# Group and calculate the mean of aggregate and refuse percentage per location for organic material
agg_subsort_o = org_sub_df.groupby("Location")["Aggregate Percent"].mean()
refuse_subsort_o = org_sub_df.groupby("Location")["Refuse Percent"].mean()
agg_mainsort_o = org_main_df.groupby("Location")["Aggregate Percent"].mean()
refuse_mainsort_o = org_main_df.groupby("Location")["Refuse Percent"].mean()


# Group and calculate the mean of aggregate and refuse percentage per location for e waste material
agg_subsort_e = ew_sub_df.groupby("Location")["Aggregate Percent"].mean()
refuse_subsort_e = ew_sub_df.groupby("Location")["Refuse Percent"].mean()
agg_mainsort_e = ew_main_df.groupby("Location")["Aggregate Percent"].mean()
refuse_mainsort_e = ew_main_df.groupby("Location")["Refuse Percent"].mean()

# Collection paper will have Average capture rate taken from Recycling_Div_Captures DataFrame
avg_cap_rate = Recycling_Div_Captures.groupby("_zone")["Paper rate"].mean()


In [23]:
# MGP collection
# 'Plastic' 'Glass' 'Metal'
# Types of Metal register in subsort
Metals_sub = cws_df.loc[(cws_df["Material Group"]=="Metal"),["Material"]]
Metals_sub = Metals_sub["Material"].unique()
set_sub = set(Metals_sub)
# Types of MGP materials register in mainsort
Metals_main = cwm_df.loc[(cwm_df["Material Group"]=="Metal"),["Material"]]
Metals_main = Metals_main["Material"].unique()
set_main = set(Metals_main)
# Mergin types of materials without duplication 
no_duplicates = list(set_main-set_sub)
Metals_sub = list(set_sub)
Metals_final= Metals_sub + no_duplicates
Metals_final

['Empty Aerosol Cans',
 'Mixed Metals',
 'Other Aluminium',
 'Appliances: Non-ferrous',
 'Appliances: Ferrous',
 'Other Ferrous Metal',
 'Aluminum Foil/Containers',
 'Steel/Tin Food Cans',
 'Other Non-ferrous',
 'Aluminum Cans',
 'Other Ferrous',
 'Other Aluminum',
 'Other Non-Ferrous',
 'Appliances: Non-Ferrous']

In [24]:
# Types of glass register in subsort
Glass_sub = cws_df.loc[(cws_df["Material Group"]=="Glass"),["Material"]]
Glass_sub = Glass_sub["Material"].unique()
set_sub = set(Glass_sub)
# Types of MGP materials register in mainsort
Glass_main = cwm_df.loc[(cwm_df["Material Group"]=="Glass"),["Material"]]
Glass_main = Glass_main["Material"].unique()
set_main = set(Glass_main)
# Mergin types of materials without duplication 
no_duplicates = list(set_main-set_sub)
Glass_sub = list(set_sub)
Glass_final= Glass_sub + no_duplicates
Glass_final

['Other Glass',
 'Green Container Glass',
 'Mixed Cullet',
 'Other Color Container Glass',
 'Brown Container Glass',
 'Clear Container Glass']

In [25]:
# Types of plastic register in subsort
Plas_sub = cws_df.loc[(cws_df["Material Group"]=="Plastic"),["Material"]]
Plas_sub = Plas_sub["Material"].unique()
set_sub = set(Plas_sub)
# Types of MGP materials register in mainsort
Plas_main = cwm_df.loc[(cwm_df["Material Group"]=="Plastic"),["Material"]]
Plas_main = Plas_main["Material"].unique()
set_main = set(Plas_main)
# Mergin types of materials without duplication 
no_duplicates = list(set_main-set_sub)
Plas_sub = list(set_sub)
Plastic_final= Plas_sub + no_duplicates
Plastic_final

['Bulk/Rigid Plastics: Toys/Housewares: #2 HDPE Natural',
 'Bulk/Rigid Plastics: Crates/Soda Carrier: #2 HDPE Pig.',
 'Bulk/Rigid Plastics: Other Durable: #2 HDPE Natural',
 'Bulk/Rigid Plastics: Other Durable: #5 PP Expanded',
 'Drinking Cups - Paper and Plastic: Plastic - Single Use: #2 HDPE Pig.',
 'Bulk/Rigid Plastics: Crates/Soda Carrier: #7 Other',
 'Thermoforms: #5 PP Expanded',
 'Rigid Plastics: Tubs & Lids: #1 PET All',
 'Drinking Cups - Paper and Plastic: Plastic - Single Use: #7 Other',
 'Drinking Cups - Paper and Plastic: Plastic - Single Use: #6 PS Rigid',
 'K-Cups: Predominanty Metal',
 'Plastic Bottles: #5 PP Expanded',
 'Bulk/Rigid Plastics: Other Durable: #6 PS Rigid',
 'Rigid Plastics: Tubs & Lids: #7 Other',
 'K-Cups: Predominanty Organic',
 'Film: Pouches',
 'Drinking Cups - Paper and Plastic: Plastic - Durable: #2 HDPE Pig.',
 '#2 HDPE Colored Bottles',
 'Thermoforms: #7 Other',
 'Bulk/Rigid Plastics: Crates/Soda Carrier: #1 PET All',
 'Bulk/Rigid Plastics: Toys/Ho

In [26]:
# Filter MGP materials subsort
MGP_sub_df = cws_df.loc[(cws_df["Material Group"]=="Glass")|(cws_df["Material Group"]=="Plastic")
                              | (cws_df["Material Group"]=="Metal"),:]
# Filter MGP materials mainsort
MGP_main_df = cwm_df.loc[(cwm_df["Material Group"]=="Glass")|(cwm_df["Material Group"]=="Plastic")
                              | (cwm_df["Material Group"]=="Metal"),:]
# Group and calculate the mean of aggregate and refuse percentage per location
MGP_agg_subsort = MGP_sub_df.groupby("Location")["Aggregate Percent"].mean()
MGP_refuse_subsort = MGP_sub_df.groupby("Location")["Refuse Percent"].mean()
MGP_agg_mainsort = MGP_main_df.groupby("Location")["Aggregate Percent"].mean()
MGP_refuse_mainsort = MGP_main_df.groupby("Location")["Refuse Percent"].mean()

# Collection MGP will have Average capture rate taken from Recycling_Div_Captures DataFrame
MGP_avg_cap_rate = Recycling_Div_Captures.groupby("_zone")["MGP rate"].mean()

In [27]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
materials_db = client["MaterialsDB"]
materials_coll = materials_db["materials"]

In [28]:
# Insert paper document in the database
materials_db.materials_coll.insert_one(
    {
        'material_group': "Paper",
        'year': 2017,
        'materials_list': paper_material_final,
        'Bronx':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Bronx,
                'Refuse Percent': refuse_subsort.Bronx,
            },
            'Capture rate': avg_cap_rate.Bronx
        },
        'Manhattan':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Manhattan,
                'Refuse Percent': refuse_subsort.Manhattan,
            },
            'Mainsort':
            {
                'Aggregate Percent': agg_mainsort.Manhattan,
                'Refuse Percent': refuse_mainsort.Manhattan,
            },
            'Capture rate': avg_cap_rate.Manhattan
        },
        'Queens':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Queens,
                'Refuse Percent': refuse_subsort.Queens,
            },
            'Capture rate': avg_cap_rate.Queens
        },
        'Brooklyn':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Brooklyn,
                'Refuse Percent': refuse_subsort.Brooklyn,
            },
            'Capture rate': avg_cap_rate.Brooklyn
        },
        'Staten Island':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort[7],
                'Refuse Percent': refuse_subsort[7],
            },
            'Capture rate': avg_cap_rate[4]
        }

    }
)



In [29]:
# inserting organic waste into mongo db

materials_db.materials_coll.insert_one(
    {
        'material_group': "Organic",
        'year': 2017,
        'materials_list': org_material_final,
        'Bronx':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_o.Bronx,
                'Refuse Percent': refuse_subsort_o.Bronx,
            },
#             'Capture rate': avg_cap_rate.Bronx
        },
#         
        'Queens':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_o.Queens,
                'Refuse Percent': refuse_subsort_o.Queens,
            },
#             'Capture rate': avg_cap_rate.Queens
        },
        'Brooklyn':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_o.Brooklyn,
                'Refuse Percent': refuse_subsort_o.Brooklyn,
            },
#             'Capture rate': avg_cap_rate.Brooklyn
        },
        'Staten Island':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_o[7],
                'Refuse Percent': refuse_subsort_o[7],
            },
#             'Capture rate': avg_cap_rate[4]
        }

    }
)

In [30]:
# inserting ewatse waste into mongo db

materials_db.materials_coll.insert_one(
    {
        'material_group': "E-waste",
        'year': 2017,
        'materials_list': org_material_final,
        'Bronx':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_e.Bronx,
                'Refuse Percent': refuse_subsort_e.Bronx,
            },
#             'Capture rate': avg_cap_rate.Bronx
        },
#         
        'Queens':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_e.Queens,
                'Refuse Percent': refuse_subsort_e.Queens,
            },
#             'Capture rate': avg_cap_rate.Queens
        },
        'Brooklyn':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_e.Brooklyn,
                'Refuse Percent': refuse_subsort_e.Brooklyn,
            },
#             'Capture rate': avg_cap_rate.Brooklyn
        },
        'Staten Island':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort_e[7],
                'Refuse Percent': refuse_subsort_e[7],
            },
#             'Capture rate': avg_cap_rate[4]
        }

    }
)

In [31]:
# Insert MGP document in the database
materials_db.materials_coll.insert_one(
    {
        'material_group': "MGP (Metal, Glass, Plastic)",
        'year': 2017,
        'materials_list':
        {
            "Metal": Metals_final,
            "Glass": Glass_final,
            "Plastic": Plastic_final
        },
        'Bronx':
        {
            'Subsort':
            {
                'Aggregate Percent': MGP_agg_subsort.Bronx,
                'Refuse Percent': MGP_refuse_subsort.Bronx,
            },
            'Mainsort':
            {
                'Aggregate Percent': MGP_agg_mainsort.Bronx,
                'Refuse Percent': MGP_refuse_mainsort.Bronx,
            },
            'Capture rate': MGP_avg_cap_rate.Bronx
        },
        'Manhattan':
        {
            'Subsort':
            {
                'Aggregate Percent': MGP_agg_subsort.Manhattan,
                'Refuse Percent': MGP_refuse_subsort.Manhattan,
            },
            'Mainsort':
            {
                'Aggregate Percent': MGP_agg_mainsort.Manhattan,
                'Refuse Percent': MGP_refuse_mainsort.Manhattan,
            },
            'Capture rate': MGP_avg_cap_rate.Manhattan
        },
        'Queens':
        {
            'Subsort':
            {
                'Aggregate Percent': MGP_agg_subsort.Queens,
                'Refuse Percent': MGP_refuse_subsort.Queens,
            },
            'Mainsort':
            {
                'Aggregate Percent': MGP_agg_mainsort.Queens,
                'Refuse Percent': MGP_refuse_mainsort.Queens,
            },
            'Capture rate': MGP_avg_cap_rate.Queens
        },
        'Brooklyn':
        {
            'Subsort':
            {
                'Aggregate Percent': MGP_agg_subsort.Brooklyn,
                'Refuse Percent': MGP_refuse_subsort.Brooklyn,
            },
            'Mainsort':
            {
                'Aggregate Percent': MGP_agg_mainsort.Brooklyn,
                'Refuse Percent': MGP_refuse_mainsort.Brooklyn,
            },
            'Capture rate': MGP_avg_cap_rate.Brooklyn
        },
        'Staten Island':
        {
            'Subsort':
            {
                'Aggregate Percent': MGP_agg_subsort[7],
                'Refuse Percent': MGP_refuse_subsort[7],
            },
            'Mainsort':
            {
                'Aggregate Percent': MGP_agg_mainsort[7],
                'Refuse Percent': MGP_refuse_mainsort[7],
            },
            'Capture rate': MGP_avg_cap_rate[4]
        }

    }
)